In [1]:
from kafka import KafkaConsumer
from kafka import TopicPartition
from datetime import datetime
import pandas as pd
from cassandra.cluster import Cluster
import json
from botocore.vendored import requests
import findspark
findspark.init()
import pyspark.sql 
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
import pandas as pd
from cassandra.cluster import Cluster
import numpy as np
from pyspark.sql.types import *

In [2]:
# #------------------------------- SPARK CONTEXT -------------------------------
sc = SparkContext('local')
spark = SparkSession(sc)
spark = SparkSession.builder.appName('CC_Fraud').getOrCreate()


In [3]:
#------------------------------- Load Model from File System -------------------------------

def MachineLearningModelLoading():
	path="/model1"
	newmodel=PipelineModel.load(path)
	return newmodel
	
newmodel = MachineLearningModelLoading()


StringIndexer_ad35a5fbb27e

In [11]:
cols = ["merchant", "category", "gender", "job"]
indexers = []
for i in range(len(cols)):
    indexers.append(StringIndexer.load('C:/Users/saritajoshi/Documents/Repos/Credit-Card-Fraud-Detection-Spark/indexers/'+cols[i]))

In [25]:
feature_columns = [col for col in df.columns if col != "is_fraud"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [30]:
#------------------------------------ Data Cleaning -------------------------------------
def predicting_status(data_all,newmodel):
	columns_new = [col.replace("-", "_") for col in data_all.columns]
	data_all = data_all.toDF(*columns_new)

	xvars = ['Unnamed: 0', 'merchant','category','amt','gender','lat','long','city_pop','job','unix_time','merch_lat','merch_long'] #all numeric
	select_cols = xvars
	data = data_all.select(select_cols)
	indexed_data = data
	for indexer in indexers:
		indexed_data = indexer.fit(indexed_data).transform(indexed_data)
	indexed_data = indexed_data.drop("merchant", "category", "gender", "job")
	indexed_data = indexed_data.withColumnRenamed("merchant_index", "merchant").withColumnRenamed("category_index", "category").withColumnRenamed("gender_index", "gender").withColumnRenamed("job_index", "job").withColumnRenamed('Unnamed: 0', '_c0')
	
	pred1 = newmodel.transform(indexed_data)
	return pred1.select("prediction")

# df = spark.read.json(sc.parallelize([dd]))
# res = predicting_status(df, newmodel)

['Unnamed: 0', 'amt', 'category', 'cc_num', 'city', 'city_pop', 'dob', 'first', 'gender', 'is_fraud', 'job', 'last', 'lat', 'long', 'merch_lat', 'merch_long', 'merchant', 'state', 'street', 'trans_date_trans_time', 'trans_num', 'unix_time', 'zip']
['_c0', 'amt', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long', 'merchant', 'category', 'gender', 'job']


In [ ]:

#------------------------------------------- CASSANDRA CONNECTION -------------------------------------------------------
cluster = Cluster()
session = cluster.connect()
session.execute("CREATE KEYSPACE IF NOT EXISTS bigdata WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3}")
session.set_keyspace("bigdata")
#session.execute("DROP TABLE testing")
session.execute("CREATE TABLE IF NOT EXISTS testing (txn_id varchar PRIMARY KEY,cc_no bigint ,txn_time varchar, year smallint, month smallint, day smallint, hour smallint, minute smallint,amount double, cc_provider int, merchant varchar,merchant_index smallint, country_name varchar, country_code varchar,country_index smallint, age int, marital_status varchar,marital_status_index smallint,gender varchar,gender_index smallint,loan varchar,loan_index smallint, status varchar,status_index smallint)")
# session.set_keyspace("mykeyspace")
# session.execute("CREATE TABLE IF NOT EXISTS training (txn_id varchar PRIMARY KEY,cc_no bigint , cc_provider int, year smallint, month smallint, day smallint, hour smallint, min smallint,txn_time varchar, merchant varchar, location varchar, country varchar, amount float, status varchar)")


In [ ]:
topic_name = 'transaction_data'
consumer = KafkaConsumer(topic_name, group_id='test-producer', bootstrap_servers=['localhost:9092'])
dd= {}
for msg in consumer:
	msg=msg.value.decode("utf-8")
	df = spark.read.json(sc.parallelize([msg]))
	status_predicted=predicting_status(df,newmodel)
	print(msg)
	if status_predicted.collect()[0][0]==0:
		status="Not Fraud"
	else:
		status="Fraud"
	msg['status'] = status
	print(msg)

	strq=" INSERT INTO testing JSON "+"'"+json.dumps(msg)+"'"
	session.execute(strq)

In [7]:
dd.keys()

dict_keys(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud'])

In [23]:
	
consumer.close()

In [5]:
spark.stop()

NameError: name 'spark' is not defined